In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
import json
from tensorflow import keras
import spacy
import nltk
import re
from nltk.stem import WordNetLemmatizer


C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [22]:
lemmatizer = WordNetLemmatizer()

def check_presence(df, col, item):
    bool_list = []
    item = clear_query(item)
    for entry in df[col]:
        bool_list.append(str(item).lower() in str(entry).lower())
    return bool_list

def clear_query(token):
        stop_words_with_numbers = [
            ("gb", ""),
            ("k", "000"),
            ("hr", ""),
            ("wh", ""),
            ("watts", ""),
            ("hrs", ""),
            ("watthr", "")
        ]
        stop_words_without_numbers = [
            ("core", ""),
            ("quad", '4'),
            ("hexa", "6"),
            ("octa", "8"),
            (" ", ""),
            (".", ""),
            (",", ""),
            ("-", ""),
            ("working", "work")
        ]

        tuples = []
        # remove non-numbered stopwords
        for word, replacement in stop_words_without_numbers:
            token = token.replace(word, replacement).strip()
        
        # remove numbered stopwords
        for word, replacement in stop_words_with_numbers:
            if len(re.findall(r"\d+" + word, token)) > 0:
                token = token.replace(word, replacement).strip()
        
        return lemmatizer.lemmatize(token)

class Chatbot:
    def __init__(self, intent_bot_path="./models/bot_V1.model", ner_bot_path="./models/spacy_V2/model-best", words_path="words.pkl", classes_path="classes.pkl", intents_path="intents.json"):
        print("loading models")
        self.intent_classifier = keras.models.load_model(intent_bot_path)
        self.nlp = spacy.load(ner_bot_path)
        self.words = pickle.load(open(words_path, "rb"))
        self.classes = pickle.load(open(classes_path, "rb"))
        self.intents = json.load(open(intents_path, "r"))
        self.lemmatizer = WordNetLemmatizer()
        self.ner_labels = [
            "MANUFACTURER",
            "MODEL",
            "CPU_BRAND",
            "CPU_SERIES",
            "CPU_MODEL",
            "CORE_COUNT",
            "CPU_GEN",
            "RAM",
            "GPU_BRAND",
            "GPU",
            "GPU_TECH",
            "GPU_SERIES",
            "VRAM",
            "SCREEN_SIZE",
            "TYPE",
            "PRICE",
            "STORAGE_TYPE",
            "STORAGE_SIZE",
            "BAT_CAP",
            "MISC",
            "PRICE_TYPE",
            "SCREEN_RESO",
            "IO",
            "OS"
        ]
        self.dataframe = pd.read_csv("./tables/products.csv")
        print("models loaded")
    
    def preprocess(self, item):
        tokens = nltk.word_tokenize(item)
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]
        return tokens

    def bag_of_words(self, sentence):
        sentence_words = self.preprocess(sentence)
        bag = [0] * len(self.words)
        for w in sentence_words:
            for i, word in enumerate(self.words):
                if word == w:
                    bag[i] = 1
        
        return np.array(bag)

    def predict_intent(self, sentence, tolerance=0.7):
        bow = self.bag_of_words(sentence)
        prediction = self.intent_classifier(np.array([bow]))[0]
        max_value = max(prediction).numpy()
        res = np.where(prediction.numpy() == max_value)[0][0]
        if res >= tolerance:
            return res
        else:
            return -1

    def ner(self, sentence):
        doc = self.nlp(sentence)
        output = {}

        for label in self.ner_labels:
            output[label] = []

        for ent in doc.ents:
            output[ent.label_].append(ent.text)
        
        return output

    def query(self, sentence):
        labels = self.ner(sentence)
        df = self.dataframe.copy(deep=True)
        response = pd.DataFrame(columns=df.columns) # self.dataframe.loc[self.dataframe[self.ner_labels[0]] == labels[self.ner_labels[0]]]
        final = []
        print(labels)
        
        for label in self.ner_labels:
            if len(labels[label]) > 0:
                print(labels[label])
                check = df if not len(response) > 0 else response
                empty = pd.DataFrame(columns=df.columns)
                for item in labels[label]:
                    temp = check.loc[check[label].apply(str).apply(str.lower).apply(clear_query).str.contains(clear_query(item.lower()))] #check.loc[check_presence(check, label, item)]
                    empty = empty.append(temp)
                response = empty.copy(deep=True)
                # return response

        for i in range(len(response)):
            final.append(response.iloc[i].to_dict())

        return final
    
    

In [23]:
bot = Chatbot()

loading models
models loaded


In [20]:
clear_query("student") # working -> work

'student'

In [27]:
(bot.query("I want to gift my friend a work laptop with 8gb ram"))

{'MANUFACTURER': [], 'MODEL': [], 'CPU_BRAND': [], 'CPU_SERIES': [], 'CPU_MODEL': [], 'CORE_COUNT': [], 'CPU_GEN': [], 'RAM': ['8gb'], 'GPU_BRAND': [], 'GPU': [], 'GPU_TECH': [], 'GPU_SERIES': [], 'VRAM': [], 'SCREEN_SIZE': [], 'TYPE': ['work'], 'PRICE': [], 'STORAGE_TYPE': [], 'STORAGE_SIZE': [], 'BAT_CAP': [], 'MISC': [], 'PRICE_TYPE': [], 'SCREEN_RESO': [], 'IO': [], 'OS': []}
['8gb']
['work']


[{'ID': 3,
  'MANUFACTURER': 'MSI',
  'MODEL': 'Modern 14 ',
  'CPU_BRAND': 'Intel',
  'CPU_SERIES': 'i5',
  'CPU_MODEL': '10210U',
  'CORE_COUNT': 4.0,
  'CPU_GEN': 10.0,
  'RAM': 8.0,
  'GPU_BRAND': 'INTEL',
  'GPU': 'UHD 630',
  'GPU_TECH': 'None',
  'GPU_SERIES': 'None',
  'VRAM': 'None',
  'SCREEN_SIZE': 14.0,
  'TYPE': 'work|student',
  'PRICE': 70000.0,
  'STORAGE_TYPE': 'SSD',
  'STORAGE_SIZE': '512GB',
  'TOTAL_STORAGE': 512.0,
  'BAT_CAP': '12',
  'IMG_1': nan,
  'IMG_2': nan,
  'IMG_3': nan,
  'MISC': 'battery|weight|light',
  'IO': nan},
 {'ID': 4,
  'MANUFACTURER': 'MSI',
  'MODEL': 'Modern 14 ',
  'CPU_BRAND': 'AMD',
  'CPU_SERIES': 'Ryzen 5',
  'CPU_MODEL': '4500U',
  'CORE_COUNT': 6.0,
  'CPU_GEN': 4.0,
  'RAM': 8.0,
  'GPU_BRAND': 'INTEL',
  'GPU': 'UHD 630',
  'GPU_TECH': 'None',
  'GPU_SERIES': 'None',
  'VRAM': 'None',
  'SCREEN_SIZE': 14.0,
  'TYPE': 'work|student',
  'PRICE': 60000.0,
  'STORAGE_TYPE': 'SSD',
  'STORAGE_SIZE': '512GB',
  'TOTAL_STORAGE': 512.0,
  

In [15]:
import pandas as pd

In [16]:
df = pd.read_clipboard()

In [ ]:
bot.ner_labels

In [21]:
outputs

{'BRAND': ['MSI'],
 'PRICE': ['70000'],
 'PRICE_TYPE': ['less than'],
 'TYPE': ['gaming'],
 'CONFIG': ['16gb'],
 'COMPONENT': ['RAM']}

In [32]:
df.columns

Index(['ID', 'MANUFACTURER', 'MODEL', 'CPU_BRAND', 'CPU_SERIES', 'CPU_MODEL',
       'CPU_GEN', 'GPU_BRAND', 'GPU', 'GPU_TECH', 'GPU_SERIES', 'RAM', 'VRAM',
       'SCREEN_SIZE', 'TYPE', 'PRICE', 'STORAGE_TYPE', 'STORAGE_SIZE',
       'TOTAL_STORAGE'],
      dtype='object')

In [30]:
label_map = {
    "BRAND" : ["MANUFACTURER", "MODEL", "CPU_BRAND", "GPU_BRAND"],
    "PRICE" : ["PRICE"],
}

In [31]:
output_df = pd.DataFrame(columns=df.columns)
for label in list(outputs.keys())[1]:
    for item in label_map[label]:
        output_df = output_df.append(df.loc[df[item] == outputs[label][0]])
    

KeyError: 'P'

In [28]:
output_df

,ID,MANUFACTURER,MODEL,CPU_BRAND,CPU_SERIES,CPU_MODEL,CPU_GEN,GPU_BRAND,GPU,GPU_TECH,GPU_SERIES,RAM,VRAM,SCREEN_SIZE,TYPE,PRICE,STORAGE_TYPE,STORAGE_SIZE,TOTAL_STORAGE
0,0,MSI,Bravo 15,AMD,Ryzen 7,4800H,4.0,AMD,RX5500M,RX,55,16.0,4,15.0,gaming,90000.0,SSD,512GB,512.0
1,1,MSI,GF75 Thin,Intel,i5,10500H,10.0,NVIDIA,GTX1650,GTX,16,8.0,4,17.0,gaming,63000.0,SSD,512GB,512.0
2,2,MSI,GE66 Raider,Intel,i7,10875H,10.0,NVIDIA,RTX2080,RTX,20,32.0,8,15.0,gaming,250000.0,SSD,2TB,2000.0
3,3,MSI,Modern 14,Intel,i5,10210U,10.0,INTEL,UHD630,None,None,8.0,None,14.0,work,70000.0,SSD,512GB,512.0
4,4,MSI,Modern 14,AMD,Ryzen 5,4500U,4.0,INTEL,UHD630,None,None,8.0,None,14.0,work,60000.0,SSD,512GB,512.0
